In [4]:
import math
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta
import ipywidgets as widgets
from ipywidgets import Layout, interact
from IPython.display import HTML, display
display(HTML("<style>.container { width:100% !important; }</style>"))

In [11]:

# static widgets
lbl_icons = widgets.Label('📊 📶 🔋',layout=Layout(height='auto', width='20%', display='flex', justify_content='flex-end', align_items='center'))
lbl_1 = widgets.Label('Total Value Chart', style=dict(font_weight='bold', font_size='18px'),layout=Layout(display='flex', justify_content='center', align_items='center', height='16%', width='auto'))
lbl_time = widgets.Label('2:23',layout=Layout(height='auto', width='10%', display='flex', justify_content='center', align_items='center'), style=dict(font_size='14px', font_weight='bold'))
icons = widgets.HBox([lbl_time, lbl_icons],layout=Layout(height='16%', width='auto', display='flex', justify_content='space-between'))
lbl_2 = widgets.Label('Total Value',layout=Layout())
lbl_3 = widgets.Label('Day Change',layout=Layout())

# dynamic widgets
lbl_marketvalue = widgets.Label('$51,749.51',layout=Layout(height='auto'), style=dict(text_color='gray', font_size='30px'))
lbl_daychange = widgets.Label('+$1,172.66 (2.32%)',layout=Layout(), style=dict(text_color='red', font_size='14px'))
lbl_period = widgets.Label('1 Month Change')
lbl_periodchange = widgets.Label('+$5,796.08 (12.61%)', style=dict(text_color='red', font_size='14px'))





def sim_drawdown(vol_pct,time_btn):
    t = times[time_btn]['n']
    v = vol_pct*max_vol*t**(-0.5)*(-2.33)
    steps = max(int(250/t),50)
    
    x = np.linspace(0,100,steps)
    
    #y = np.random.normal(loc=math.log(1+v)/steps,scale=-v/steps**0.75,size=steps)
    #y = np.cumsum(y)-y[0]
    #y = y*y/y[-1]
    #y += 1
    #y *= mv_start
    
    #ysmooth = np.linspace(mv_start,mv_start*(1+v), steps)

    y = np.random.normal(loc=math.log(1+v)/steps,scale=-v/(steps**0.5),size=steps-1)
    y -= (sum(y)-math.log(1+v))/(steps)
    y = np.exp(y)
    y = np.insert(y,0,mv_start)
    y = np.cumprod(y)

    # plotting style
    fig, ax = plt.subplots(facecolor='#111111', figsize=(5,5.5)) #6.8
    ax.plot(x,y, color='#009EDC')
    
    #ax.plot(x,ysmooth)
    
    ax.fill_between(x,y, alpha=0.1, facecolor='#009EDC', hatch='|')
    ax.set_facecolor('#111111')
    ax.set_ylim(min(y)*0.999, max(y)*1.001)
    flt_labels = ax.get_yticks()
    ax.set_yticks(flt_labels)
    ax.set_yticklabels([f'${round(i,-1):.0f}' if i<1000 else (f'${i/1000:.1f}K' if i<1000000 else f'${i/1000000:.2f}M') for i in flt_labels])
    ax.set_xticks([0,20,40,60,80])
    ax.set_xticklabels(times[time_btn]['lbl'],  ha='left')
    ax.yaxis.set_label_position('right')
    ax.tick_params(axis='both', colors='gray', length=0)
    ax.yaxis.tick_right()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

    # update other widgets
    lbl_marketvalue.value = f'${y[-1]:,.2f}'
    lbl_period.value = times[time_btn]['chglbl']
    
    lbl_periodchange.value = f'-${abs(y[-1]-y[0]):,.2f} (-{100*abs(y[-1]/y[0]-1):.2f}%)'
    lbl_daychange.value = lbl_periodchange.value if time_btn=='1 D' else (lambda a,b: f'-${b-a:,.2f} ({(a/b-1)*100:.2f}%)')(y[-1],y[-1]*np.random.uniform(1.005,1.02))

    

times = {'1 D':{'n':250, 'lbl':['6:30','8:00','9:30','11:00','12:30'], 'chglbl': '1 Day Change'},
         '1 M':{'n':12, 'lbl':[(date.today()-timedelta(days=(5-d)*6.1)).strftime('%b %d') for d in range(5)], 'chglbl': '1 Month Change'},
         '3 M':{'n':4, 'lbl':[(date.today()-timedelta(days=(5-d)*18.25)).strftime('%b %d') for d in range(5)], 'chglbl': '3 Month Change'},
         '6 M':{'n':2, 'lbl':[(date.today()-timedelta(days=(5-d)*36.5)).strftime('%b') for d in range(5)], 'chglbl': '6 Month Change'},
         '1 Y':{'n':1, 'lbl':[(date.today()-timedelta(days=(5-d)*73)).strftime('%b \'%y') for d in range(5)], 'chglbl': '1 Year Change'}}
results = {'1 D':{'a': None,'b': None},
           '1 M':{'a': None,'b': None},
           '3 M':{'a': None,'b': None},
           '6 M':{'a': None,'b': None},
           '1 Y':{'a': None,'b': None}}

max_vol = 0.3
mv_start = 2000000


time_buttons = widgets.ToggleButtons(options=['1 D', '1 M', '3 M', '6 M', '1 Y'], layout=Layout(width='100%'), style=dict(button_width='14%'))
vol_slider = widgets.FloatSlider(value=0.07, min=0.01, max=1, step=0.01, readout=False, layout=Layout(width='22vw'))
price_chart = widgets.interactive_output(sim_drawdown,{'vol_pct': vol_slider, 'time_btn': time_buttons})


# container widgets
value_box = widgets.VBox([lbl_2, lbl_marketvalue, lbl_3, lbl_daychange],layout=Layout(height='74%', width='auto'))
header = widgets.VBox([icons, lbl_1, value_box], layout=Layout(height='24%', width='auto'))
body = widgets.VBox([price_chart], layout=Layout(height='61%', width='auto'))
footer_values = widgets.VBox([lbl_period, lbl_periodchange], layout=Layout(height='70%', width='auto'))


footer_buttons = widgets.HBox([time_buttons],layout=Layout(height='30%', width='auto'))
footer = widgets.VBox([footer_values, footer_buttons], layout=Layout(height='15%', width='auto'))
test = widgets.VBox([header, body, footer],layout=Layout(height='100%', width='22vw', border='2px solid gray')) #22vw


def record_sigma1(b):
    v = vol_slider.value*max_vol*times[time_buttons.value]['n']**(-0.5)*(-2.33)
    results[time_buttons.value]['a'] = v/(-1.65)
    
def record_sigma2(b):
    v = vol_slider.value*max_vol*times[time_buttons.value]['n']**(-0.5)
    results[time_buttons.value]['b'] = v
    
    
sigma1 = widgets.Button(button_style='warning', layout=Layout(width='30px'))
sigma2 = widgets.Button(button_style='danger', layout=Layout(width='30px'))
sigma1.on_click(record_sigma1)
sigma2.on_click(record_sigma2)

confirm = widgets.Button(button_style='info', layout=Layout(width='30px'))

phone = widgets.VBox([test, widgets.HBox([vol_slider, sigma1, sigma2, confirm], layout=Layout(width='22vw'))], #22vw
                     layout=Layout(display='flex', flex_flow='column', align_items='center'))

def show_volchart(b):
    main.children = [phone, vol_box]
    annl_btn = widgets.ToggleButton(description='Toggle Annualized', value=False)
    vol_chart = widgets.interactive_output(interact_volchart,{'annl': annl_btn})
    vol_box.children = [vol_chart, annl_btn]
    
    return

def interact_volchart(annl):
    fig, ax = plt.subplots()
    
    x = [1/250,1/12,1/4,1/2,1]
    if annl:
        sig1 = [results[i]['a']*times[i]['n']**0.5 for i in results]
        sig2 = [results[i]['b']*times[i]['n']**0.5 for i in results]
        avg = [0.5*(np.average(sig1)+np.average(sig2))]*len(x)
        ax.plot(x,avg)
        
    else:
        sig1 = [results[i]['a'] for i in results]
        sig2 = [results[i]['b'] for i in results]
    ax.scatter(x,sig1)
    ax.scatter(x,sig2)
    
    ax.set_ylim(0,0.4)
    

confirm.on_click(show_volchart)

vol_box = widgets.VBox(layout=Layout(height='100%', display='flex', flex_flow='both', justify_content='center', align_items='center'))

main = widgets.HBox([phone], layout=Layout(display='flex', flex_flow='row', justify_content='space-around', height='100%')) #display='flex', flex_flow='column', align_items='stretch',




display(main)


In [836]:
results

{'1 D': {'a': 0.005894485558553862, 'b': 0.006451046426743494},
 '1 M': {'a': 0.03301918675883578, 'b': 0.03204293994002423},
 '3 M': {'a': 0.06142727272727274, 'b': 0.058499999999999996},
 '6 M': {'a': 0.0808801592808104, 'b': 0.0742462120245875},
 '1 Y': {'a': 0.10590909090909091, 'b': 0.111}}

In [837]:
sig1 = [results[i]['a']*times[i]['n']**0.5 for i in results]
sig2 = [results[i]['b']*times[i]['n']**0.5 for i in results]